In [ ]:
#|default_exp filter

In [ ]:
#|export
from __future__ import annotations
import json

from nbprocess.read import *
from nbprocess.sync import *
from nbprocess.process import *
from nbprocess.processors import *
from fastcore.utils import *
from fastcore.script import call_parse,bool_arg

# quarto

> Filter for quarto processing

In [ ]:
#|export
class FilterDefaults:
    def _nothing(self): return []
    xtra_procs=xtra_preprocs=xtra_postprocs=_nothing
    
    def base_procs(self):
        return [strip_ansi, meta_, hide_line, filter_stream_, clean_magics,
                bash_identify, rm_header_dash, rm_export, clean_show_doc, remove_]
    def base_preprocs(self): return [insert_warning]
    def base_postprocs(self): return []

    def procs(self):
        "Processors for export"
        return self.base_procs + self.xtra_procs

    def preprocs(self):
        "Preprocessors for export"
        return self.base_preprocs + self.xtra_preprocs

    def postprocs(self):
        "Postprocessors for export"
        return self.base_postprocs + self.xtra_postprocs

In [ ]:
#|export
@call_parse
def filter_nb(
    nb_txt:str=None  # Notebook text (uses stdin if not provided)
):
    filt = get_config().get('exporter', FilterDefaults)()
    printit = False
    if not nb_txt:
        nb_txt = sys.stdin.read()
        printit = True
    nb = dict2nb(json.loads(nb_txt))
    nbp = NBProcessor(nb=nb, procs=exp.procs(), preprocs=exp.preprocs(), postprocs=exp.postprocs())
    nbp.process()
    res = nb2str(nb)
    if printit: print(res)
    else: return res

In [ ]:
#|export
@call_parse
def create_quarto(
    path:str=None, # path to notebooks
    symlinks:bool=False, # follow symlinks?
    file_glob:str='*.ipynb', # Only include files matching glob
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None, # Skip files matching regex
    skip_folder_re:str='^[_.]' # Skip folders matching regex
):
    path = get_config().path("nbs_path") if not path else Path(path)
    files = globtastic(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re,
                       folder_re=folder_re, skip_file_glob=skip_file_glob,
                       skip_file_re=skip_file_re, skip_folder_re=skip_folder_re
                      ).sorted().map(Path)
    yml = L((path/'_quarto.yml').read_text().splitlines(True))
    re_contents = re.compile('^\s+contents:\s*$').search
    yml = yml[yml.argfirst(re_contents)+1:]
    return '\n'.join(f'- {o.relative_to(path)}' for o in files)

In [ ]:
print(create_quarto())

- 00_read.ipynb
- 01_maker.ipynb
- 02_process.ipynb
- 03_export.ipynb
- 04_sync.ipynb
- 05_merge.ipynb
- 06_clean.ipynb
- 07_processors.ipynb
- 08_filter.ipynb
- index.ipynb


## Export -

In [ ]:
#skip
from nbprocess.export import nbs_export
nbs_export()